# import

In [1]:
import math
import os
import pandas as pd
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras

# constants

In [2]:
BATCH_SIZE = 100
EPOCHS = 20
LEARNING_RATE = 0.1
THRESHOLDS = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

# fetch

In [3]:
working_df = pd.read_csv("../data/is_in.csv")
working_df.head()

,x,y,is_in
0,-0.342671,-0.377219,1
1,-2.189994,-3.914946,0
2,-0.087206,-3.781911,0
3,2.124552,-2.604753,0
4,3.807756,-3.584953,0


# prepare

In [4]:
working_df["cross"] = working_df["x"]**2 + working_df["y"]**2
working_df = working_df[["cross", "is_in"]]
working_df.head()

,cross,is_in
0,0.259717,1
1,20.122880,0
2,14.310453,0
3,11.298459,0
4,27.350895,0


In [5]:
length_all = len(working_df)
length_train = math.trunc(length_all * 0.8)
end_train = length_train - 1
start_test = length_train
train_df = working_df.iloc[:end_train]
test_df = working_df.iloc[start_test:]

# train

In [6]:
inputs = keras.Input(shape=(1,))
outputs = keras.layers.Dense(1, activation=keras.activations.sigmoid)(inputs)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=LEARNING_RATE),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.FalseNegatives(THRESHOLDS),
        keras.metrics.FalsePositives(THRESHOLDS),
        keras.metrics.TrueNegatives(THRESHOLDS),
        keras.metrics.TruePositives(THRESHOLDS),
    ],
)

history = model.fit(
    x=train_df["cross"].values,
    y=train_df["is_in"].values,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - false_negatives: 125.8148 - false_positives: 3.5000 - loss: 0.9445 - true_negatives: 306.2778 - true_positives: 52.8518   
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - false_negatives: 54.2963 - false_positives: 13.7593 - loss: 0.2988 - true_negatives: 298.6852 - true_positives: 121.7037
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - false_negatives: 42.3704 - false_positives: 14.9630 - loss: 0.2611 - true_negatives: 294.0370 - true_positives: 137.0741
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - false_negatives: 32.8148 - false_positives: 14.7037 - loss: 0.2252 - true_negatives: 298.7408 - true_positives: 142.1852
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - false_negatives: 29.6296 - false_positives: 14.2222 - loss: 0.2115 - true_negatives: 297.2222 - true_positives: 147.3704
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - false_negatives: 23.5741 - false_positives: 12.9815 - loss: 0.1822 - true_negatives: 299.9074

# evaluate

In [8]:
weight = model.get_weights()[0][0][0]
bias = model.get_weights()[1][0]
print(f"weight: {weight}")
print(f"bias: {bias}")

evaluation = model.evaluate(
    x=test_df["cross"].values,
    y=test_df["is_in"].values,
    batch_size=BATCH_SIZE,
    return_dict=True,
    verbose=0,
)
metrics_df = pd.DataFrame(columns=["metric"] + THRESHOLDS)
tp = evaluation["true_positives"].numpy().tolist()
metrics_df.loc[len(metrics_df)] = ["TP"] + tp
fp = evaluation["false_positives"].numpy().tolist()
metrics_df.loc[len(metrics_df)] = ["FP"] + fp
fn = evaluation["false_negatives"].numpy().tolist()
metrics_df.loc[len(metrics_df)] = ["FN"] + fn
tn = evaluation["true_negatives"].numpy().tolist()
metrics_df.loc[len(metrics_df)] = ["TN"] + tn
accuracy = []
for i in range(len(THRESHOLDS)):
    accuracy.append((tp[i] + tn[i]) / (tp[i] + tn[i] + fp[i] + fn[i]))
metrics_df.loc[len(metrics_df)] = ["Accuracy"] + accuracy
fpr = []
for i in range(len(THRESHOLDS)):
    fpr.append(fp[i] / (fp[i] + tn[i]))
metrics_df.loc[len(metrics_df)] = ["FPR"] + fpr
tpr = []
for i in range(len(THRESHOLDS)):
    tpr.append(tp[i] / (tp[i] + fn[i]))
metrics_df.loc[len(metrics_df)] = ["TPR"] + tpr
print(metrics_df)

weight: -1.159325361251831
bias: 7.938429355621338
     metric      0.2         0.3      0.4         0.5         0.6         0.7
0        TP   75.000   73.000000   72.000   68.000000   65.000000   61.000000
1        FP   11.000    7.000000    0.000    0.000000    0.000000    0.000000
2        FN    0.000    2.000000    3.000    7.000000   10.000000   14.000000
3        TN  114.000  118.000000  125.000  125.000000  125.000000  125.000000
4  Accuracy    0.945    0.955000    0.985    0.965000    0.950000    0.930000
5       FPR    0.088    0.056000    0.000    0.000000    0.000000    0.000000
6       TPR    1.000    0.973333    0.960    0.906667    0.866667    0.813333
